## * Setup steps

In [ ]:
!pip install -q -U pydata-google-auth google-cloud-bigquery pyarrow

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['font.family'] = 'serif'
%config InlineBackend.figure_format = 'svg'
import os
hostname = os.popen('hostname').read().strip()

PROJECT_ID = "mlforhc2019-worker0-c1fd" # Modify this line to your worker project ID

from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)

%load_ext google.cloud.bigquery

# Example Study: Cohort of patients with diabetes and are managing with lifestyle changes

In this sample notebook, we will extract data related to our cohort of interest from the MGI database. 

We will use the following pragmatic definition of diabetes, modified from [Narges Razavian et al., 2015](https://doi.org/10.1089/big.2015.0020). A patient is considered to have diabetes if:

1. (a diagnosis code related to Diabetes under the Elixhauser comorbidity index); **OR** (HbA1C value > 6.5%)

    **AND**

2. use of a diabetes medication, i.e. with a therapeutic class of "antihyperglycemics"

The onset time of diabetes for a patient is defined as the earliest time that any of the criteria above are met. 

## 1. Cohort discovery

### 1.1 Decide what to look for and verity table content

In [ ]:
%%bigquery --project $PROJECT_ID
# Display the list of ICD codes used for Elixhauser comorbidity definition
SELECT * FROM `mlforhc2019-data-ee97.mlforhc2019_data_data_bq.ComorbidityAlgorithmCodes` 
WHERE Algorithm = 'Elixhauser' AND Comorbidity LIKE "Diabetes%" LIMIT 1000

In [ ]:
%%bigquery --project $PROJECT_ID
# Display the list of antihyperglycemic drugs
SELECT DISTINCT(MO.MedicationName)
FROM `mlforhc2019-data-ee97.mlforhc2019_data_data_bq.MedicationOrders` MO  
WHERE
  upper(MO.TherapeuticClass) like '%ANTIHYPERGLYCEMICS%'

### 1.2 Extract cohort

In [ ]:
%%bigquery df_cohort --project $PROJECT_ID
WITH elixhauserPatients AS
(
  SELECT 
    Diag.PatientID,
    MIN(coalesce(Diag.DiagnosisObservationDate,Diag.ActivityDate)) as `dbDate`
  FROM 
  `mlforhc2019-data-ee97.mlforhc2019_data_data_bq.ComorbidityAlgorithmCodes` CAC
  JOIN
  `mlforhc2019-data-ee97.mlforhc2019_data_data_bq.Diagnoses` Diag on CAC.Lexicon = Diag.Lexicon  and CAC.DiagnosisCode  = Diag.TermCodeMapped 
  WHERE
    lower(CAC.Comorbidity) LIKE '%diabetes%'
    AND
    lower(CAC.Algorithm  ) LIKE '%elix%'
  GROUP BY
    Diag.PatientID
),
labPatients AS
(
  SELECT 
    LR.PatientID,
    MIN(LR.Observation_Date) AS `dbDate`
  FROM `mlforhc2019-data-ee97.mlforhc2019_data_data_bq.LabResults` LR 
  WHERE 
    LOWER(result_name) like '%a1c%' 
    AND SAFE_CAST(Value AS FLOAT64) > 6.5
  GROUP BY
    LR.PatientID
),
medPatients AS
(
  SELECT
    MO.PatientID,
    MIN(MO.OrderDate) AS `dbDate`
  FROM `mlforhc2019-data-ee97.mlforhc2019_data_data_bq.MedicationOrders` MO  
  WHERE
    UPPER(MO.TherapeuticClass) LIKE '%ANTIHYPERGLYCEMICS%' 
  GROUP BY
    MO.PatientID
),
allPatients AS
(
  SELECT *
  FROM elixhauserPatients ep
  UNION ALL
  SELECT * 
  FROM labPatients lp
)

SELECT
  ap.PatientID,
  min(ap.dbDate) AS `earliestDate`
FROM
  allPatients ap
  JOIN
  medPatients mp ON ap.patientID = mp.patientID
GROUP BY
  ap.PatientID

In [ ]:
print('Number of patients in cohort:', len(df_cohort))

In [ ]:
df_cohort.head()

In [ ]:
IDs = df_cohort['PatientID']

## 2. Demographics

Distributions of gender and age

In [ ]:
%%bigquery df_demog --project $PROJECT_ID
SELECT * FROM `mlforhc2019-data-ee97`.mlforhc2019_data_data_bq.PatientInfo

### 2.1 Gender distribution

In [ ]:
df_pop = df_demog[df_demog['PatientID'].isin(IDs)]
df_pop.groupby('GenderName')[['PatientID']].count()

### 2.2 Age distribution

In [ ]:
%%bigquery df_age --project $PROJECT_ID
SELECT PatientID, MAX(AgeInYears) AS Age # Get age at most recent encounter
FROM `mlforhc2019-data-ee97`.mlforhc2019_data_data_bq.Encounter 
GROUP BY PatientID

In [ ]:
df_pop_age = df_age[df_age['PatientID'].isin(IDs)].copy()
df_pop_age.describe()

# Replace redacted ages (>89) with 91, the median of those ages
df_pop_age.loc[df_pop_age['Age'] == 300, 'Age'] = 91

# Plot a histogram
df_pop_age.hist(bins=np.arange(15,100,2))
_, max_ = plt.ylim()
x = df_pop_age['Age']
plt.axvline(x.median(), color='k', linestyle='dashed', linewidth=1)
plt.text(x.median() - x.median()/3, max_ - max_/10, 'Median: {:.2f}'.format(x.median()))
plt.show()

## 3. Retrive additional data and save as tables

In [ ]:
# Helper function to upload pandas dataframe to BigQuery
dataset_id="mlforhc2019_worker0_data_bq"
dataset = bigquery.Dataset("{}.{}".format(PROJECT_ID, dataset_id))
dataset.location = "US"
client.get_dataset(dataset)

def upload_table(df, table_name):
    # Upload a pandas dataframe into your BigQuery dataset
    job = client.load_table_from_dataframe(df, dataset.table(table_name), location="US")
    job.result()  # Waits for table load to complete.
    assert job.state == "DONE"
    print("Done: upload to", table_name, "in", dataset_id)

### 3.1 Save the cohort of interest

In [ ]:
# Choose a unique table name, in this case use the hostname of this VM
upload_table(df_cohort, table_name = hostname + '_' + 'Cohort')

### 3.2 Top 10 procedures within cohort

In [ ]:
q = """
SELECT
  Pr.TermCodeMapped,
  Pr.termNameMapped,
  COUNT(DISTINCT Pr.PatientID) AS `numPatients`
FROM
  `{}.{}.{}` P
JOIN
  `mlforhc2019-data-ee97.mlforhc2019_data_data_bq.Procedures` Pr
ON
  P.patientID = Pr.patientID
GROUP BY
  Pr.TermCodeMapped,
  Pr.termNameMapped
ORDER BY
  `numPatients` DESC
LIMIT
  10
""".format(PROJECT_ID, dataset.dataset_id, hostname + '_' + 'Cohort')
job_config = bigquery.QueryJobConfig()
df_proc = client.query(q, job_config=job_config).to_dataframe()
df_proc

### 3.3 Height, Weight, Body-mass index (BMI)

In [ ]:
# Save the query results to a BigQuery table in your BigQuery dataset directly
# Without downloading and uploading

# Delete table if exists
table_name = hostname + '_' + 'BMI'
client.delete_table(dataset.table(table_name), not_found_ok=True)

# Run query
q = """
SELECT P.PatientID, B.EncounterID, B.MedianHeightCm, B.MedianWeightKG, B.BMI
FROM `{}.{}.{}` P
  JOIN `mlforhc2019-data-ee97.mlforhc2019_data_data_bq.Encounter` E on P.PatientID = E.PatientID
  JOIN `mlforhc2019-data-ee97.mlforhc2019_data_data_bq.EncounterBmi` B on B.EncounterID  = E.EncounterID 
""".format(PROJECT_ID, dataset.dataset_id, hostname + '_' + 'Cohort')
job_config = bigquery.QueryJobConfig()
job_config.destination = dataset.table(table_name) # Set destination table
job = client.query(q, job_config=job_config)

# Waits for the query to finish
job.result()
print('Done: saving', table_name, 'in', dataset.dataset_id)

In [ ]:
q = """SELECT * FROM `{}.{}.{}` LIMIT 100""".format(PROJECT_ID, dataset.dataset_id, hostname + '_' + 'BMI')
job_config = bigquery.QueryJobConfig()
client.query(q, job_config=job_config).to_dataframe()

### 3.4 Social history

Is this patient ever a smoker?

In [ ]:
# Save the query results to a BigQuery table in your BigQuery dataset directly
# Without downloading and uploading

# Delete table if exists
table_name = hostname + '_' + 'Smoker'
client.delete_table(dataset.table(table_name), not_found_ok=True)

# Run query
q = """
WITH
  patientStatuses AS(
  SELECT
    SH.PatientID,
    CASE
      WHEN SH.SmokingStatusMapped = 'Former' OR SH.SmokingStatusMapped = 'Current' THEN 1
    ELSE
    0
  END
    AS `hasEverSmoked`
  FROM
    `{}.{}.{}` P
  JOIN
    `mlforhc2019-data-ee97.mlforhc2019_data_data_bq.SocialHistory` SH
  ON
    P.PatientID = SH.PatientID ),
  patientEverSmoked AS (
  SELECT
    patientID,
    MAX(hasEverSmoked) AS `hasEverSmoked`
  FROM
    patientStatuses
  GROUP BY
    PatientID )
SELECT
  *
FROM
  patientEverSmoked
""".format(PROJECT_ID, dataset.dataset_id, hostname + '_' + 'Cohort')
job_config = bigquery.QueryJobConfig()
job_config.destination = dataset.table(table_name) # Set destination table
job = client.query(q, job_config=job_config)

# Waits for the query to finish
job.result()
print('Done: saving', table_name, 'in', dataset.dataset_id)

In [ ]:
q = """SELECT * FROM `{}.{}.{}` LIMIT 100""".format(PROJECT_ID, dataset.dataset_id, hostname + '_' + 'Smoker')
job_config = bigquery.QueryJobConfig()
client.query(q, job_config=job_config).to_dataframe()

### 3.5 Vital signs

In [ ]:
# Save the query results to a BigQuery table in your BigQuery dataset directly
# Without downloading and uploading

# Delete table if exists
table_name = hostname + '_' + 'VitalSigns'
client.delete_table(dataset.table(table_name), not_found_ok=True)

# Run query
q = """
with locationTagRanking as
(
  select
    LocationID,
    case
      when lower(LocationTag) like '%emergency%' then 3
      when lower(LocationTag) like '%inpatient%' then 2
      else 1  -- "Other"
    end as `tagRank`
  from
    `mlforhc2019-data-ee97.mlforhc2019_data_data_bq.LocationTags` 
),
locationClass as
(
  select
    LocationID,
    max(tagRank) as `tagClass`
  from
    locationTagRanking
  Group by
    LocationID
)
SELECT SV.PatientID,
  SV.EncounterID,
  SV.ObservationDateTime,
  SV.BPDiaInvasive,
  SV.BPSysInvasive,
  SV.BPDiaNonInvasive,
  SV.BPSysNonInvasive,
  SV.HeartRate,
  case
    when lc.tagClass = 3 then 'Emergency'
    when lc.tagClass = 2 then 'Inpatient'
    when lc.tagClass = 1 then 'Other'
    else 'Unknown'
   end as `patientSetting`  
FROM 
  `{}.{}.{}` P
join
  `mlforhc2019-data-ee97.mlforhc2019_data_data_bq.NursingStandardVitalSigns` SV on P.PatientID = SV.PatientID
left outer join
  `mlforhc2019-data-ee97.mlforhc2019_data_data_bq.EncounterLocations` el on SV.EncounterID  = el.EncounterID and SV.ObservationDateTime  between el.StartDate  and el.EndDate 
left outer join
  locationClass lc on el.LocationID = lc.LocationID
where
  SV.BPDiaInvasive is not null
  or SV.BPSysInvasive is not null
  or SV.BPDiaNonInvasive is not null
  or SV.BPSysNonInvasive is not null
  or SV.HeartRate is not null
  or SV.Temperature is not null
""".format(PROJECT_ID, dataset.dataset_id, hostname + '_' + 'Cohort')
job_config = bigquery.QueryJobConfig()
job_config.destination = dataset.table(table_name) # Set destination table
job = client.query(q, job_config=job_config)

# Waits for the query to finish
job.result()
print('Done: saving', table_name, 'in', dataset.dataset_id)

In [ ]:
q = """SELECT * FROM `{}.{}.{}` LIMIT 100""".format(PROJECT_ID, dataset.dataset_id, hostname + '_' + 'VitalSigns')
job_config = bigquery.QueryJobConfig()
client.query(q, job_config=job_config).to_dataframe()

### 3.6 Lab results

In [ ]:
# Save the query results to a BigQuery table in your BigQuery dataset directly
# Without downloading and uploading

# Delete table if exists
table_name = hostname + '_' + 'Labs'
client.delete_table(dataset.table(table_name), not_found_ok=True)

# Run query
q = """
SELECT 
  LR.PatientID ,
  LR.EncounterID,
  LR.Order_Date ,
  LR.Order_Code ,
  LR.Order_Name ,
  LR.Collection_Date ,
  LR.Observation_Date ,
  LR.Result_Code ,
  LR.Result_Name ,
  LR.LOINC,
  LR.Value,
  SAFE_CAST(LR.Value as FLOAT64) as `Value_Numeric`,
  LR.UNit,
  LR.Range
FROM 
  `{}.{}.{}` P
  join
  `mlforhc2019-data-ee97.mlforhc2019_data_data_bq.LabResults` LR on P.PatientID = LR.PatientID
WHERE
  Value is not null
""".format(PROJECT_ID, dataset.dataset_id, hostname + '_' + 'Cohort')
job_config = bigquery.QueryJobConfig()
job_config.destination = dataset.table(table_name) # Set destination table
job = client.query(q, job_config=job_config)

# Waits for the query to finish
job.result()
print('Done: saving', table_name, 'in', dataset.dataset_id)

In [ ]:
q = """SELECT * FROM `{}.{}.{}` LIMIT 100""".format(PROJECT_ID, dataset.dataset_id, hostname + '_' + 'Labs')
job_config = bigquery.QueryJobConfig()
client.query(q, job_config=job_config).to_dataframe()

### 3.7 Medication Administrations

In [ ]:
# Save the query results to a BigQuery table in your BigQuery dataset directly
# Without downloading and uploading

# Delete table if exists
table_name = hostname + '_' + 'Meds'
client.delete_table(dataset.table(table_name), not_found_ok=True)

# Run query
q = """
SELECT
  MA.*
FROM
  `{}.{}.{}` P
JOIN
  `mlforhc2019-data-ee97.mlforhc2019_data_data_bq.MedicationAdministrations` MA
ON
  P.PatientID = MA.PatientID
""".format(PROJECT_ID, dataset.dataset_id, hostname + '_' + 'Cohort')
job_config = bigquery.QueryJobConfig()
job_config.destination = dataset.table(table_name) # Set destination table
job = client.query(q, job_config=job_config)

# Waits for the query to finish
job.result()
print('Done: saving', table_name, 'in', dataset.dataset_id)

In [ ]:
q = """SELECT * FROM `{}.{}.{}` LIMIT 100""".format(PROJECT_ID, dataset.dataset_id, hostname + '_' + 'Meds')
job_config = bigquery.QueryJobConfig()
client.query(q, job_config=job_config).to_dataframe()

### 3.8 Encounter location type counts

In [ ]:
# Save the query results to a BigQuery table in your BigQuery dataset directly
# Without downloading and uploading

# Delete table if exists
table_name = hostname + '_' + 'LocationCounts'
client.delete_table(dataset.table(table_name), not_found_ok=True)

# Run query
q = """
SELECT 
  P.PatientID,
  L.LocationTag,
  count(distinct E.EncounterID) as `numEncounters`
FROM 
  `{}.{}.{}` P
  join
  `mlforhc2019-data-ee97.mlforhc2019_data_data_bq.Encounter` E on P.PatientID = E.PatientID
  join
  `mlforhc2019-data-ee97.mlforhc2019_data_data_bq.EncounterLocations` EL on E.EncounterID = EL.EncounterID
  join
  `mlforhc2019-data-ee97.mlforhc2019_data_data_bq.LocationTags` L on EL.LocationID = L.LocationID
GROUP BY
  P.PatientID,
  L.LocationTag
""".format(PROJECT_ID, dataset.dataset_id, hostname + '_' + 'Cohort')
job_config = bigquery.QueryJobConfig()
job_config.destination = dataset.table(table_name) # Set destination table
job = client.query(q, job_config=job_config)

# Waits for the query to finish
job.result()
print('Done: saving', table_name, 'in', dataset.dataset_id)

In [ ]:
q = """SELECT * FROM `{}.{}.{}` LIMIT 100""".format(PROJECT_ID, dataset.dataset_id, hostname + '_' + 'LocationCounts')
job_config = bigquery.QueryJobConfig()
client.query(q, job_config=job_config).to_dataframe()

### 3.9 Orders related to a dietitian

In [ ]:
# Save the query results to a BigQuery table in your BigQuery dataset directly
# Without downloading and uploading

# Delete table if exists
table_name = hostname + '_' + 'DietitianOrders'
client.delete_table(dataset.table(table_name), not_found_ok=True)

# Run query
q = """
select 
  O.*
from
`{}.{}.{}` P
join
`mlforhc2019-data-ee97.mlforhc2019_data_data_bq.Non_MedicationOrders` O on P.PatientID = O.PatientID 
where
(
  lower(ordername) like '%dietitian%' 
  or lower(ordertype) like '%dietitian%'
)
""".format(PROJECT_ID, dataset.dataset_id, hostname + '_' + 'Cohort')
job_config = bigquery.QueryJobConfig()
job_config.destination = dataset.table(table_name) # Set destination table
job = client.query(q, job_config=job_config)

# Waits for the query to finish
job.result()
print('Done: saving', table_name, 'in', dataset.dataset_id)

In [ ]:
q = """SELECT * FROM `{}.{}.{}` LIMIT 100""".format(PROJECT_ID, dataset.dataset_id, hostname + '_' + 'DietitianOrders')
job_config = bigquery.QueryJobConfig()
client.query(q, job_config=job_config).to_dataframe()

## 4. View final tables in the dataset

In [ ]:
%bigquery df_tables --project $PROJECT_ID 
SELECT
 * EXCEPT(is_typed)
FROM
 mlforhc2019_worker0_data_bq.INFORMATION_SCHEMA.TABLES

In [ ]:
# Obtain a list of tables that you created
df_tables[df_tables['table_name'].str.startswith(hostname)]

# 5. Clean-up

In [ ]:
## !!DANGER!! ##
# Delete your existing tables
dataset_id="mlforhc2019_worker0_data_bq"
dataset = bigquery.Dataset("{}.{}".format(PROJECT_ID, dataset_id))
dataset.location = "US"
client.get_dataset(dataset)
for tn in df_tables[df_tables['table_name'].str.startswith(hostname)]['table_name']:
    client.delete_table(dataset.table(tn), not_found_ok=True)